<a href="https://colab.research.google.com/github/WeiHongWi/CatvsDog/blob/main/CatvsDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir("/content/drive/My Drive")
!ls

In [ ]:
!pip install torch torchvision

In [ ]:
cd examples/imagenet

[Errno 2] No such file or directory: 'examples/imagenet'
/content


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import time
import torch
import torch.nn as nn
from pathlib import Path
from torch.optim import Adam
from torchvision import datasets ,models,transforms
from sklearn.model_selection import train_test_split
from torch.nn import Linear, CrossEntropyLoss, Module


path_train = r'/content/drive/MyDrive/examples/imagenet/train'
TRAIN =Path(path_train)

#將數據集的圖片一律改成224*224，並丟入function transform成 size:224*224,tensor,
#Tensor()將images的每個pixels轉成-1.0~1.0之間的數值
transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])


In [ ]:
#將train set分類
train_data = datasets.ImageFolder(TRAIN, transform=transforms)
print(train_data.class_to_idx)

#將train分成80%train set和20%validation set
print(len(train_data))
train_size = int(0.8 * len(train_data))
valid_size = int(0.2 * len(train_data))
print(train_size)

#根據size大小運用函數來隨機分配給數據集
train_data , valid_data = torch.utils.data.random_split(train_data,[train_size,valid_size])

#將Dataset裏頭的data先打亂在分配成batch
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=100,shuffle=True)

for i,item in enumerate(valid_loader):
  l = item
  print('data:',data)
  print('label:',label)


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#構建 resnet18
from torchvision.models.resnet import resnet18
resnet = resnet18()

#從print的最後一層輸出的維度為1000，但我們需要的只有貓跟狗，所以只用2種就好，修改resnet
fc_feature = resnet.fc.in_features
resnet.fc = Linear(fc_feature,2)

#print(resnet.fc)
#print(resnet)


In [ ]:
#設定訓練參數
lr = 0.01
epochs = 15
input_size = 224
batch_size = 100

In [ ]:
#用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = resnet.to(device) #將resnet丟進GPU
#Train
def train(net,optimizer,device,criterion,train_loader,epoch):

  net.train()
  batch_number = len(train_loader)
  run_loss = 0.0
  correct = 0
  traintime = 0.0

  for i,data in enumerate(train_loader,start = 1):
    start = time.time()
    #將input的資料要送入GPU
    inputs ,labels = data
    inputs ,labels = inputs.to(device) , labels.to(device)
    
    outputs = net(inputs) #順向傳遞
    
    loss = criterion(outputs,labels) #計算loss
    
    optimizer.zero_grad()#若沒有這個function，則梯度自動累加
    loss.backward()#反向傳遞
    optimizer.step()#更新所有參數
    
    transpre = outputs.data.max(dim=1)[1]
    total = outputs.shape[0]
    correct += (transpre == labels).sum().item()
    run_loss += loss.item()
    end = time.time()
    e = end-start
    traintime = traintime + e
    #print(traintime)
    #print(total)
    
    
    if i%10 == 0:
      #print(correct)
      total = total*10
      acc = 100*correct/total
      print('Epoch:[{}][{}/{}] Time:{} loss:{:3f} Acc:{} '.format(epoch+1,i,batch_number,traintime,run_loss/10,acc))
      run_loss = 0.0
      correct = 0
      traintime = 0
#Validation
def valid(net,device,valid_loader,criterion):
  net.eval() #關閉dropout，訓練好的網路才不會改變
  batch_number = len(valid_loader)
  val_run_loss = 0.0
  vcorrect = 0
  val_total = 0
  testtime = 0
  for i,target in enumerate(valid_loader,start = 1):
    with torch.no_grad(): #停止gradient計算，加速計算
      start = time.time()
      images,labels = target
      images,labels = images.to(device),labels.to(device)
      
      valid_outputs = net(images)
      validpre = valid_outputs.data.max(dim=1)[1]

      vcorrect +=(validpre == labels).sum().item()
      val_run_loss += criterion(valid_outputs,labels).item()
      val_total += valid_outputs.shape[0]
      
      end = time.time()
      e = end-start
      testtime = testtime + e
      
      if i%10 == 0:
        #print(vcorrect)
        accv = 100*vcorrect/val_total
        print('Test:[{}/{}] Time:{} loss:{:.6f} Acc:{} '.format(i,batch_number,testtime,val_run_loss/10,accv))
        val_run_loss = 0.0
        vcorrect = 0
        testtime = 0
        val_total = 0

#開始訓練
criterion = CrossEntropyLoss()
optimizer = torch.optim.SGD(net.fc.parameters(),lr=lr)
for epoch in range(epochs):
  print('{}th Train'.format(epoch+1))
  train(net,optimizer,device,criterion,train_loader,epoch)
  valid(net,device,valid_loader,criterion)